# Read Dataset

In [1]:
import sys, os
sys.path.append(os.path.split(os.getcwd())[0])
import utils
import glob

In [2]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

dataset_dir = "../dataset/"
files = glob.glob(dataset_dir+"p*.txt")
files.sort(key=natural_keys)
docs = [utils.read_file(x) for x in files]
files

['../dataset/p0.txt',
 '../dataset/p1.txt',
 '../dataset/p2.txt',
 '../dataset/p3.txt',
 '../dataset/p4.txt',
 '../dataset/p5.txt',
 '../dataset/p6.txt',
 '../dataset/p7.txt',
 '../dataset/p8.txt',
 '../dataset/p9.txt',
 '../dataset/p10.txt',
 '../dataset/p11.txt',
 '../dataset/p12.txt',
 '../dataset/p13.txt',
 '../dataset/p14.txt',
 '../dataset/p15.txt',
 '../dataset/p16.txt',
 '../dataset/p17.txt',
 '../dataset/p18.txt',
 '../dataset/p19.txt',
 '../dataset/p20.txt',
 '../dataset/p21.txt',
 '../dataset/p22.txt',
 '../dataset/p23.txt',
 '../dataset/p24.txt',
 '../dataset/p25.txt',
 '../dataset/p26.txt',
 '../dataset/p27.txt',
 '../dataset/p28.txt',
 '../dataset/p29.txt',
 '../dataset/p30.txt',
 '../dataset/p31.txt',
 '../dataset/p32.txt',
 '../dataset/p33.txt',
 '../dataset/p34.txt',
 '../dataset/p35.txt',
 '../dataset/p36.txt',
 '../dataset/p37.txt',
 '../dataset/p38.txt',
 '../dataset/p39.txt',
 '../dataset/p40.txt',
 '../dataset/p41.txt',
 '../dataset/p42.txt',
 '../dataset/p43.txt'

In [ ]:
# docs

In [3]:
import spacy
nlp = spacy.load('en')

In [4]:
%%time
processed_docs = []    
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)
    
# This method takes roughly ~ 10 mins

CPU times: user 11min 27s, sys: 2min 57s, total: 14min 24s
Wall time: 8min 43s


In [5]:
docs = processed_docs
del processed_docs

In [6]:
len(docs)

319

In [7]:
# Compute bigrams.
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/Users/nus/git/tpms/venv/lib/python2.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.

from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 20
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.

In [9]:
# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
len(corpus)

In [10]:
author2doc = utils.read_json_file("../dataset/authors.json")
author2doc

{u'Anthony_Tung': [84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103],
 u'Beng_Chin_OOI': [194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  165,
  166,
  206,
  207,
  208,
  209,
  210,
  211,
  212],
 u'Bingsheng_He': [47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58],
 u'Bryan_Low': [25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46],
 u'David_Hsu': [117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135],
 u'David_S._Rosenblum': [213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  174,
  223],
 u'Gim_Hee_Lee': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 u'Haizhou_Li': [104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 u'Harold_Soh': [312, 313, 314, 315, 316, 317, 318, 1

In [ ]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                author2doc=author2doc, chunksize=2000, passes=1, eval_every=0, \
                iterations=1, random_state=1)

In [11]:
%%time
# model_list = []

from gensim.models import AuthorTopicModel

model_list = []
num_topics = 15

for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=num_topics, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

#     print("Perplexity = %.3e" %model.bound(chunk=corpus, ))

CPU times: user 1min 2s, sys: 2.37 s, total: 1min 4s
Wall time: 1min 4s


In [12]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: -1.557e+01


In [14]:
# Save model.
model.save('model.atmodel')

In [13]:
model.show_topics(num_topics=num_topics)

[(0,
  u'0.024*"face" + 0.023*"image" + 0.011*"facial" + 0.010*"occlusion" + 0.010*"age" + 0.009*"layer" + 0.007*"object" + 0.007*"detection" + 0.006*"deep" + 0.005*"cnn"'),
 (1,
  u'0.021*"image" + 0.017*"fault" + 0.016*"signal" + 0.015*"position" + 0.014*"light" + 0.011*"motion" + 0.011*"detection" + 0.009*"constraint" + 0.008*"pass" + 0.008*"robot"'),
 (2,
  u'0.010*"graph" + 0.009*"node" + 0.008*"u" + 0.008*"gpu" + 0.008*"transaction" + 0.008*"query" + 0.007*"concept" + 0.007*"memory" + 0.006*"block" + 0.006*"record"'),
 (3,
  u'0.012*"query" + 0.012*"image" + 0.009*"record" + 0.008*"index" + 0.008*"cluster" + 0.007*"attribute" + 0.006*"similarity" + 0.006*"edge" + 0.006*"selection" + 0.005*"tuple"'),
 (4,
  u'0.026*"consumer" + 0.021*"image" + 0.016*"goal" + 0.013*"package" + 0.010*"simulation" + 0.009*"concept" + 0.009*"period" + 0.009*"product" + 0.007*"price" + 0.007*"self"'),
 (5,
  u'0.015*"variable" + 0.015*"formula" + 0.012*"count" + 0.010*"sat" + 0.010*"q" + 0.010*"countin

In [23]:
topic_labels = ['Computer Vision', 'Motion Detection', 'Database System', \
                'Clustering', 'Simulation', 'Sampling & Counting', \
                'Gaussian Process', 'Speech Processing', 'NLP', \
                'Reinforcement Learning', 'Recommender System', \
                'Collaborative AI Systems', 'Pose estimation', 'Media Search & Retrieval', \
                'POMDP']

In [26]:
for topic in model.show_topics(num_topics=num_topics):
    print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print

Label: Computer Vision
Words: face image facial occlusion age layer object detection deep cnn 

Label: Motion Detection
Words: image fault signal position light motion detection constraint pass robot 

Label: Database System
Words: graph node u gpu transaction query concept memory block record 

Label: Clustering
Words: query image record index cluster attribute similarity edge selection tuple 

Label: Simulation
Words: consumer image goal package simulation concept period product price self 

Label: Sampling & Counting
Words: variable formula count sat q counting constraint benchmark independent rf 

Label: Gaussian Process
Words: gp gaussian agent q u proc in_proc dt z robot 

Label: Speech Processing
Words: speech language pp tool signal alignment acoustic speaker gan deep 

Label: NLP
Words: language translation english rule in_proceeding score sentence relation workshop chinese 

Label: Reinforcement Learning
Words: decision action concept node variable policy transition selection

In [31]:
sorted(model['Bingsheng_He'], key=lambda x: x[1], reverse=True)

[(2, 0.9478781368818591),
 (5, 0.01842768025316355),
 (1, 0.013419519264571128),
 (6, 0.010673665067926285)]

In [36]:
from pprint import pprint

def show_author(name):
    print('\n%s' % name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    sorted_topics = sorted(model[name], key=lambda x: x[1], reverse=True)
    pprint([(topic_labels[topic[0]], topic[1]) for topic in sorted_topics])

In [37]:
for author in author2doc.keys():
    show_author(author)


Sanjay_Jain
('Docs:', [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Topics:
[('Simulation', 0.9999642100381495)]

Shuicheng_Yan
('Docs:', [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83])
Topics:
[('Computer Vision', 0.9999779291279178)]

Kay_Chen_Tan
('Docs:', [105, 158, 159, 108, 160, 115])
Topics:
[('Speech Processing', 0.515746570543134),
 ('Collaborative AI Systems', 0.24062687254342172),
 ('Reinforcement Learning', 0.04410997519241778),
 ('Computer Vision', 0.03790093564026754),
 ('Simulation', 0.032284912061083686),
 ('Recommender System', 0.022506248098537615),
 ('POMDP', 0.02056526886975451),
 ('Clustering', 0.019054743514355044),
 ('Gaussian Process', 0.01877395697699087),
 ('Motion Detection', 0.018671484159551256),
 ('Sampling & Counting', 0.016362649905206604)]

Tat_Seng_CHUA
('Docs:', [173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193])
Topics:
[('Recommender System', 0.6382006733874321),
 ('Me

# Model Evaluation

In [ ]:
from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
# Compute the per-word bound.
# Number of words in corpus.
corpus_words = sum(cnt for document in model.corpus for _, cnt in document)

# Compute bound and divide by number of words.
perwordbound = model.bound(model.corpus, author2doc=model.author2doc, \
                           doc2author=model.doc2author) / corpus_words
print(perwordbound)

In [ ]:
%time top_topics = model.top_topics(model.corpus)

In [ ]:
top_topics

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

In [ ]:
# Tell Bokeh to display plots inside the notebook.
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

# Radius of each point corresponds to the number of documents attributed to that author.
scale = 0.1
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

# Similarity queries

In [38]:
from gensim.similarities import MatrixSimilarity

# Generate a similarity object for the transformed corpus.
index = MatrixSimilarity(model[list(model.id2author.values())])

# Get similarities to some author.
author_name = 'Bryan_Low'
sims = index[model[author_name]]

In [39]:
# Make a function that returns similarities based on the Hellinger distance.

from gensim import matutils
import pandas as pd

# Make a list of all the author-topic distributions.
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    '''Get similarity between two vectors'''
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    '''Get similarity of vector to all authors.'''
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author names, and author sizes.
    Return `top_n` authors as a dataframe.
    
    '''
    
    # Get similarities.
    sims = get_sims(model.get_author_topics(name))

    # Arrange author names, similarities, and author sizes in a list of tuples.
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # Make dataframe and retrieve top authors.
    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [41]:
get_table('Bryan_Low')

,Author,Score,Size
3,Bryan_Low,1.000000,22
23,Wee_Sun_Lee,0.551585,19
12,Lee_Wee_Sun,0.527027,19
5,David_S._Rosenblum,0.523337,12
10,Kay_Chen_Tan,0.519392,6
8,Harold_Soh,0.517168,9
21,Teck_Khim_Ng,0.514421,5
2,Bingsheng_He,0.514305,12
22,Terence_Sim,0.501558,12
16,Rudy_Setiono,0.500997,8
